In [1]:
import time
from itertools import product

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


In [2]:
# Set seeds
torch.manual_seed(0)
np.random.seed(0)

In [3]:
synthetic_calls_path = '../data/binom_synthetic_calls.csv'
synthetic_puts_path = '../data/binom_synthetic_puts.csv'

In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    
    return df

In [5]:
synthetic_calls = pd.read_csv(synthetic_calls_path, index_col=0)
synthetic_puts = pd.read_csv(synthetic_puts_path, index_col=0)

synthetic_calls = reduce_mem_usage(synthetic_calls)
synthetic_puts = reduce_mem_usage(synthetic_puts)

In [6]:
synthetic_options = pd.concat([synthetic_calls, synthetic_puts], axis=0)
synthetic_options = shuffle(synthetic_options, random_state=0)
synthetic_options = synthetic_options.reset_index()
synthetic_options = synthetic_options.drop('index', axis=1)

In [7]:
synthetic_options.head(5)

,Price,Strike,Type,Vol,Interest Rate,Time to Expiration,Option Price
0,100,97.0,C,0.799805,0.070007,0.600098,27.125000
1,100,101.0,P,0.500000,0.099976,0.600098,12.671875
2,100,90.0,P,0.300049,0.059998,0.899902,4.707031
3,100,104.0,P,0.899902,0.090027,0.899902,30.390625
4,100,144.0,C,1.000000,0.099976,0.899902,28.078125


# Preprocessing

In [8]:
synthetic_options = pd.get_dummies(synthetic_options, prefix='', prefix_sep='')

In [9]:
input_sc = StandardScaler()
output_sc = StandardScaler()
input_data = input_sc.fit_transform(synthetic_options.drop('Option Price', axis=1))
output_data = output_sc.fit_transform(synthetic_options['Option Price'].values.reshape(-1, 1))

train_size = 0.9
last_train_idx = int(np.round(len(input_data) * train_size))

X_train = input_data[0:last_train_idx]
X_test = input_data[last_train_idx:]

y_train = output_data[0:last_train_idx]
y_test = output_data[last_train_idx:]

In [10]:
X_train = Variable(torch.Tensor(X_train))
X_test = Variable(torch.Tensor(X_test))

y_train = Variable(torch.Tensor(y_train))
y_test = Variable(torch.Tensor(y_test))

 # Model

In [11]:
CUDA = torch.cuda.is_available()
device = 'cuda:0' if CUDA else 'cpu'

In [12]:
class ResBlock(nn.Module):

  def __init__(self, module):
    super(ResBlock, self).__init__()
    self.module = module

  def forward(self, x):
    return self.module(x) + x

In [13]:
class HiddenLayer(nn.Module):

  def __init__(self, layer_size, act_fn):
      super(HiddenLayer, self).__init__()
      
      if act_fn == 'ReLU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.ReLU())
      elif act_fn == 'LeakyReLU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.LeakyReLU())
      elif act_fn == 'ELU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.ELU())
    
  def forward(self, x):
    return self.layer(x)

In [14]:
class Net(nn.Module):

  def __init__(self, input_size, output_size, hidden_size, num_layers, act_fn):
    super(Net, self).__init__()
    self.input_size = input_size
    self.output_size = output_size
    self.hidden_size = hidden_size

    if act_fn == 'ReLU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ReLU())
    elif act_fn == 'LeakyReLU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.LeakyReLU())
    elif act_fn == 'ELU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ELU())

    self.hidden_layers_list = []

    for i in range(num_layers // 2):
      self.hidden_layers_list.append(
          ResBlock(
            nn.Sequential(
                HiddenLayer(self.hidden_size, act_fn),
                HiddenLayer(self.hidden_size, act_fn)
            )
        )
      )

    self.hidden_layers = nn.Sequential(*self.hidden_layers_list)

    self.net = nn.Sequential(
        self.initial_layer,
        self.hidden_layers,
        nn.Linear(self.hidden_size, self.output_size)
    )
  
  def forward(self, x):
    return self.net(x)

In [15]:
def init_weights(m, init_m: str):

  @torch.no_grad()
  def init_uniform(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.uniform_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_normal(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.normal_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_xuniform(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_xnormal(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.xavier_normal_(m.weight)
      m.bias.data.fill_(0.01)

  if init_m == 'uniform':
    m.apply(init_uniform)
  elif init_m == 'normal':
    m.apply(init_normal)
  elif init_m == 'xaiver uniform':
    m.apply(init_xuniform)
  elif init_m == 'xavier normal':
    m.apply(init_xnormal)

## Hyperparameter options

In [16]:
hidden_size = [200, 400, 600]
n_layers = [4, 6, 8]
act_fun = ['ReLU', 'LeakyReLU', 'ELU']
init_methods = ['xavier uniform', 'xavier normal']
epochs = 25
n_folds = 5

In [17]:
cv_params = list(product(hidden_size,
                         n_layers,
                         act_fun,
                         init_methods))
n_cv_params = len(cv_params)
n_cv_params

54

In [18]:
sample_proportion = 0.6
sample_size = int(sample_proportion * n_cv_params)

cv_param_sample = np.random.choice(list(range(n_cv_params)),
                                     size=int(sample_size),
                                     replace=False)
cv_params_ = [cv_params[i] for i in cv_param_sample]
print('# CV parameters:', len(cv_params_))

# CV parameters: 32


# Training

In [19]:
input_size = 7
output_size = 1
batch_size = 1208
lr = 1e-4

loss_fn = nn.MSELoss()

In [20]:
class OptDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

  def __len__(self):
    return len(self.X)

In [21]:
def evaluate(model, loss_fn, X_val, y_val):
  model.eval()
  with torch.no_grad():
    out = model(X_val)
    loss = loss_fn(out, y_val)
    return loss.item()

In [22]:
crossval_result = pd.DataFrame(columns=
                               ['hidden_size',
                                'n_layers',
                                'act_fun',
                                'init_methods',
                                'mean_val_result',
                                'std_val_result'])

In [23]:
def cross_validation(
  cv_params_,
  epochs,
  n_folds,
  batch_size,
  X,
  y,
  loss_fn
):
  crossval_result = pd.DataFrame(columns=
                               ['hidden_size',
                                'n_layers',
                                'act_fun',
                                'init_methods',
                                'mean_val_result',
                                'std_val_result'])

  for h_size, n_layers, act, init_m in cv_params_:
    model = Net(input_size, output_size, h_size, n_layers, act).to(device)
    init_weights(model, init_m)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    c_val_size = len(X) // n_folds

    validation_losses = []
    cv_res = {
        'hidden_size': h_size,
        'n_layers': n_layers,
        'act_fun': act,
        'init_methods': init_m
    }

    print('Model: ', cv_res)

    for i in range(n_folds):
      model.train()

      X_train = torch.cat((X[0:c_val_size*i], X[c_val_size*(i+1):]))
      y_train = torch.cat((y[0:c_val_size*i], y[c_val_size*(i+1):]))
      X_val = X[c_val_size*i:c_val_size*(i+1)]
      y_val = y[c_val_size*i:c_val_size*(i+1)]

      for epoch in range(epochs):

        for batch, batch_labels in DataLoader(OptDataset(
            X_train, 
            y_train), 
            batch_size=batch_size):
          out = model(batch.to(device))
          optimizer.zero_grad()

          loss = loss_fn(out, batch_labels.to(device))
          loss.backward()
          optimizer.step()

        validation_losses.append(evaluate(model, 
                                          loss_fn, 
                                          X_val.to(device), 
                                          y_val.to(device)))
        print('fold: ', i + 1 ,', epoch: ', epoch + 1, ', val loss: ', validation_losses[-1])

    validation_losses = np.array(validation_losses)
    cv_res['mean_val_result'] = validation_losses.mean()
    cv_res['std_val_result'] = validation_losses.std()
    print('Model results: ', cv_res, '\n')
    crossval_result = crossval_result.append(cv_res, ignore_index=True)


  return crossval_result

In [24]:
crossval_results = cross_validation(cv_params_, 
                                    epochs, 
                                    n_folds, 
                                    batch_size,
                                    X_train,
                                    y_train,
                                    loss_fn)

Model:  {'hidden_size': 600, 'n_layers': 8, 'act_fun': 'ELU', 'init_methods': 'xavier normal'}
fold:  1 , epoch:  1 , val loss:  0.004184442572295666
fold:  1 , epoch:  2 , val loss:  0.001227384665980935
fold:  1 , epoch:  3 , val loss:  0.0015790387988090515
fold:  1 , epoch:  4 , val loss:  0.00038043083623051643
fold:  1 , epoch:  5 , val loss:  0.0003624866367317736
fold:  1 , epoch:  6 , val loss:  0.00028280678088776767
fold:  1 , epoch:  7 , val loss:  0.0010647728340700269
fold:  1 , epoch:  8 , val loss:  0.00032244017347693443
fold:  1 , epoch:  9 , val loss:  0.00016348068311344832
fold:  1 , epoch:  10 , val loss:  0.0005539879784919322
fold:  1 , epoch:  11 , val loss:  0.00019334722310304642
fold:  1 , epoch:  12 , val loss:  0.0009476420818828046
fold:  1 , epoch:  13 , val loss:  0.0010139707010239363
fold:  1 , epoch:  14 , val loss:  0.0016273148357868195
fold:  1 , epoch:  15 , val loss:  0.00017711175314616412
fold:  1 , epoch:  16 , val loss:  0.000102713216620031

fold:  1 , epoch:  13 , val loss:  6.318851956166327e-05
fold:  1 , epoch:  14 , val loss:  2.0704970665974542e-05
fold:  1 , epoch:  15 , val loss:  3.3405063732061535e-05
fold:  1 , epoch:  16 , val loss:  7.820063183316961e-05
fold:  1 , epoch:  17 , val loss:  0.00025096317403949797
fold:  1 , epoch:  18 , val loss:  2.0367167962831445e-05
fold:  1 , epoch:  19 , val loss:  2.4869646949809976e-05
fold:  1 , epoch:  20 , val loss:  8.727824024390429e-05
fold:  1 , epoch:  21 , val loss:  2.6011586669483222e-05
fold:  1 , epoch:  22 , val loss:  0.00011127620382467285
fold:  1 , epoch:  23 , val loss:  2.6818608603207394e-05
fold:  1 , epoch:  24 , val loss:  1.7316519006271847e-05
fold:  1 , epoch:  25 , val loss:  5.472290649777278e-05
fold:  2 , epoch:  1 , val loss:  0.00010275805834680796
fold:  2 , epoch:  2 , val loss:  1.7212822058354504e-05
fold:  2 , epoch:  3 , val loss:  1.416673057974549e-05
fold:  2 , epoch:  4 , val loss:  1.8451924916007556e-05
fold:  2 , epoch:  5 , 

fold:  2 , epoch:  2 , val loss:  1.159082239610143e-05
fold:  2 , epoch:  3 , val loss:  9.692519597592764e-06
fold:  2 , epoch:  4 , val loss:  1.0515496796870138e-05
fold:  2 , epoch:  5 , val loss:  9.716713975649327e-06
fold:  2 , epoch:  6 , val loss:  6.135208877822151e-06
fold:  2 , epoch:  7 , val loss:  1.2552953194244765e-05
fold:  2 , epoch:  8 , val loss:  5.644363409373909e-05
fold:  2 , epoch:  9 , val loss:  2.062359453702811e-05
fold:  2 , epoch:  10 , val loss:  2.7827649319078773e-05
fold:  2 , epoch:  11 , val loss:  1.8319253285881132e-05
fold:  2 , epoch:  12 , val loss:  1.9400720702833496e-05
fold:  2 , epoch:  13 , val loss:  5.754057383455802e-06
fold:  2 , epoch:  14 , val loss:  7.725527211732697e-06
fold:  2 , epoch:  15 , val loss:  4.020398046122864e-05
fold:  2 , epoch:  16 , val loss:  9.869249879557174e-06
fold:  2 , epoch:  17 , val loss:  1.835109833336901e-05
fold:  2 , epoch:  18 , val loss:  6.290745204751147e-06
fold:  2 , epoch:  19 , val loss: 

fold:  2 , epoch:  16 , val loss:  1.6725447494536638e-05
fold:  2 , epoch:  17 , val loss:  1.1329660082992632e-05
fold:  2 , epoch:  18 , val loss:  3.343671414768323e-05
fold:  2 , epoch:  19 , val loss:  3.340365583426319e-05
fold:  2 , epoch:  20 , val loss:  8.479610187350772e-06
fold:  2 , epoch:  21 , val loss:  1.0786008715513162e-05
fold:  2 , epoch:  22 , val loss:  7.143617040128447e-06
fold:  2 , epoch:  23 , val loss:  2.3277048967429437e-05
fold:  2 , epoch:  24 , val loss:  1.9494404114084318e-05
fold:  2 , epoch:  25 , val loss:  1.0860469046747312e-05
fold:  3 , epoch:  1 , val loss:  6.672677045571618e-06
fold:  3 , epoch:  2 , val loss:  6.508530987048289e-06
fold:  3 , epoch:  3 , val loss:  9.940004019881599e-06
fold:  3 , epoch:  4 , val loss:  7.879998520365916e-06
fold:  3 , epoch:  5 , val loss:  8.648913535580505e-06
fold:  3 , epoch:  6 , val loss:  1.5004959095676895e-05
fold:  3 , epoch:  7 , val loss:  2.4938532078522258e-05
fold:  3 , epoch:  8 , val los

fold:  3 , epoch:  5 , val loss:  0.00021887135517317802
fold:  3 , epoch:  6 , val loss:  0.0001276827388210222
fold:  3 , epoch:  7 , val loss:  6.152045534690842e-05
fold:  3 , epoch:  8 , val loss:  7.441697380272672e-05
fold:  3 , epoch:  9 , val loss:  0.00010634979844326153
fold:  3 , epoch:  10 , val loss:  5.4751300922362134e-05
fold:  3 , epoch:  11 , val loss:  1.6675321603543125e-05
fold:  3 , epoch:  12 , val loss:  2.3750064428895712e-05
fold:  3 , epoch:  13 , val loss:  4.908817572868429e-05
fold:  3 , epoch:  14 , val loss:  2.0656982087530196e-05
fold:  3 , epoch:  15 , val loss:  4.756159978569485e-05
fold:  3 , epoch:  16 , val loss:  2.458396556903608e-05
fold:  3 , epoch:  17 , val loss:  3.485723209450953e-05
fold:  3 , epoch:  18 , val loss:  4.467489634407684e-05
fold:  3 , epoch:  19 , val loss:  3.063111216761172e-05
fold:  3 , epoch:  20 , val loss:  2.5302209905930795e-05
fold:  3 , epoch:  21 , val loss:  2.7519768991624005e-05
fold:  3 , epoch:  22 , val 

...

fold:  3 , epoch:  25 , val loss:  4.187091326457448e-05
fold:  4 , epoch:  1 , val loss:  3.85563907912001e-05
fold:  4 , epoch:  2 , val loss:  4.8259898903779685e-05
fold:  4 , epoch:  3 , val loss:  3.580267002689652e-05
fold:  4 , epoch:  4 , val loss:  4.7057470510480925e-05
fold:  4 , epoch:  5 , val loss:  2.7675498131429777e-05
fold:  4 , epoch:  6 , val loss:  3.0850449547870085e-05
fold:  4 , epoch:  7 , val loss:  5.180664447834715e-05
fold:  4 , epoch:  8 , val loss:  4.343307591625489e-05
fold:  4 , epoch:  9 , val loss:  4.856322630075738e-05
fold:  4 , epoch:  10 , val loss:  4.093089228263125e-05
fold:  4 , epoch:  11 , val loss:  2.479388058418408e-05
fold:  4 , epoch:  12 , val loss:  2.6433379389345646e-05
fold:  4 , epoch:  13 , val loss:  2.9633980375365354e-05
fold:  4 , epoch:  14 , val loss:  4.960068326909095e-05
fold:  4 , epoch:  15 , val loss:  2.0436302293092012e-05
fold:  4 , epoch:  16 , val loss:  2.3494811102864332e-05
fold:  4 , epoch:  17 , val loss:

fold:  4 , epoch:  14 , val loss:  5.334925663191825e-05
fold:  4 , epoch:  15 , val loss:  1.3055963790975511e-05
fold:  4 , epoch:  16 , val loss:  3.177545659127645e-05
fold:  4 , epoch:  17 , val loss:  2.1771613319288008e-05
fold:  4 , epoch:  18 , val loss:  1.5791856640134938e-05
fold:  4 , epoch:  19 , val loss:  1.3307469089340884e-05
fold:  4 , epoch:  20 , val loss:  0.00011406256089685485
fold:  4 , epoch:  21 , val loss:  4.772954707732424e-05
fold:  4 , epoch:  22 , val loss:  8.146541222231463e-05
fold:  4 , epoch:  23 , val loss:  6.579519686056301e-05
fold:  4 , epoch:  24 , val loss:  2.3045982743497007e-05
fold:  4 , epoch:  25 , val loss:  8.44261194288265e-06
fold:  5 , epoch:  1 , val loss:  4.1428640543017536e-05
fold:  5 , epoch:  2 , val loss:  1.9503420844557695e-05
fold:  5 , epoch:  3 , val loss:  9.160751687886659e-06
fold:  5 , epoch:  4 , val loss:  1.6469864931423217e-05
fold:  5 , epoch:  5 , val loss:  1.878477087302599e-05
fold:  5 , epoch:  6 , val l

fold:  5 , epoch:  3 , val loss:  1.3437806046567857e-05
fold:  5 , epoch:  4 , val loss:  5.999774657539092e-06
fold:  5 , epoch:  5 , val loss:  6.815914275648538e-06
fold:  5 , epoch:  6 , val loss:  2.0813336959690787e-05
fold:  5 , epoch:  7 , val loss:  1.04509426819277e-05
fold:  5 , epoch:  8 , val loss:  6.6681118369160686e-06
fold:  5 , epoch:  9 , val loss:  5.583111033047317e-06
fold:  5 , epoch:  10 , val loss:  9.3320086307358e-05
fold:  5 , epoch:  11 , val loss:  6.388183464878239e-06
fold:  5 , epoch:  12 , val loss:  8.890546268958133e-06
fold:  5 , epoch:  13 , val loss:  6.518696409330005e-06
fold:  5 , epoch:  14 , val loss:  5.4360829381039366e-05
fold:  5 , epoch:  15 , val loss:  8.393461939704139e-06
fold:  5 , epoch:  16 , val loss:  8.440250894636847e-06
fold:  5 , epoch:  17 , val loss:  8.241205250669736e-06
fold:  5 , epoch:  18 , val loss:  1.7108835891122e-05
fold:  5 , epoch:  19 , val loss:  4.7179546527331695e-06
fold:  5 , epoch:  20 , val loss:  8.7

fold:  5 , epoch:  17 , val loss:  4.8409343435196206e-05
fold:  5 , epoch:  18 , val loss:  2.7483396479510702e-05
fold:  5 , epoch:  19 , val loss:  4.6788868530711625e-06
fold:  5 , epoch:  20 , val loss:  4.404319952300284e-06
fold:  5 , epoch:  21 , val loss:  5.827518634760054e-06
fold:  5 , epoch:  22 , val loss:  4.20197920902865e-06
fold:  5 , epoch:  23 , val loss:  2.9480603188858368e-05
fold:  5 , epoch:  24 , val loss:  4.2083620428456925e-06
fold:  5 , epoch:  25 , val loss:  3.571508204913698e-05
Model results:  {'hidden_size': 600, 'n_layers': 6, 'act_fun': 'ELU', 'init_methods': 'xavier uniform', 'mean_val_result': 9.108797466251417e-05, 'std_val_result': 0.000209640278954211} 

Model:  {'hidden_size': 400, 'n_layers': 4, 'act_fun': 'LeakyReLU', 'init_methods': 'xavier uniform'}
fold:  1 , epoch:  1 , val loss:  0.000314035831252113
fold:  1 , epoch:  2 , val loss:  0.00013095929170958698
fold:  1 , epoch:  3 , val loss:  0.00011370726133463904
fold:  1 , epoch:  4 , v

fold:  1 , epoch:  1 , val loss:  0.0021956481505185366
fold:  1 , epoch:  2 , val loss:  0.0007669304613955319
fold:  1 , epoch:  3 , val loss:  0.0011890748282894492
fold:  1 , epoch:  4 , val loss:  0.0012491738889366388
fold:  1 , epoch:  5 , val loss:  0.0005461494438350201
fold:  1 , epoch:  6 , val loss:  0.00019976316252723336
fold:  1 , epoch:  7 , val loss:  0.0009781289845705032
fold:  1 , epoch:  8 , val loss:  0.0001903671509353444
fold:  1 , epoch:  9 , val loss:  0.0001526595588074997
fold:  1 , epoch:  10 , val loss:  0.0002948542241938412
fold:  1 , epoch:  11 , val loss:  0.0001709515490802005
fold:  1 , epoch:  12 , val loss:  0.00010046294482890517
fold:  1 , epoch:  13 , val loss:  0.00036627304507419467
fold:  1 , epoch:  14 , val loss:  0.00012706322013400495
fold:  1 , epoch:  15 , val loss:  0.00014690094394609332
fold:  1 , epoch:  16 , val loss:  0.0004330354568082839
fold:  1 , epoch:  17 , val loss:  0.0006296837236732244
fold:  1 , epoch:  18 , val loss:  

In [25]:
crossval_results

,hidden_size,n_layers,act_fun,init_methods,mean_val_result,std_val_result
0,600,8,ELU,xavier normal,0.000228,0.000452
1,400,8,LeakyReLU,xavier normal,0.000039,0.000059
2,600,8,ReLU,xavier uniform,0.000028,0.000059
3,400,6,LeakyReLU,xavier uniform,0.000024,0.000030
4,200,6,ELU,xavier normal,0.000115,0.000409
5,200,4,LeakyReLU,xavier uniform,0.000039,0.000126
6,400,8,LeakyReLU,xavier uniform,0.000022,0.000029
7,600,6,ReLU,xavier uniform,0.000024,0.000029
8,600,6,LeakyReLU,xavier normal,0.000044,0.000068
9,400,8,ReLU,xavier uniform,0.000026,0.000040


In [27]:
hyperparam_res = crossval_results.groupby(by=['hidden_size', 'n_layers', 'act_fun', 'init_methods'])

In [28]:
hyperparam_res.apply(lambda x : x.nsmallest(3, 'mean_val_result')).drop(['hidden_size', 'n_layers', 'act_fun', 'init_methods'], axis=1)

mean_val_result  \
hidden_size n_layers act_fun   init_methods                         
200         4        ELU       xavier normal  26         0.000194   
                               xavier uniform 10         0.000192   
                     LeakyReLU xavier uniform 5          0.000039   
            6        ELU       xavier normal  4          0.000115   
                               xavier uniform 11         0.000159   
                     ReLU      xavier normal  17         0.000070   
            8        ELU       xavier uniform 28         0.000125   
                     LeakyReLU xavier normal  25         0.000052   
                               xavier uniform 18         0.000029   
400         4        ELU       xavier uniform 13         0.000124   
                     LeakyReLU xavier uniform 30         0.000024   
                     ReLU      xavier uniform 22         0.000019   
            6        ELU       xavier normal  27         0.000136   
                               xavier uniform 12         0.000125   
                     LeakyReLU xavier normal  19         0.000034   
                               xavier uniform 3          0.000024   
            8        ELU       xavier normal  20         0.000164   
                               xavier uniform 24         0.000117   
                     LeakyReLU xavier normal  1          0.000039   
                               xavier uniform 6          0.000022   
                     ReLU      xavier normal  14         0.000036   
                               xavier uniform 9          0.000026   
600         4        LeakyReLU xavier uniform 15         0.000024   
                     ReLU      xavier normal  16         0.000025   
            6        ELU       xavier uniform 29         0.000091   
                     LeakyReLU xavier normal  8          0.000044   
                     ReLU      xavier uniform 7          0.000024   
            8        ELU       xavier normal  0          0.000228   
                               xavier uniform 31         0.000124   
                     LeakyReLU xavier normal  23         0.000033   
                     ReLU      xavier normal  21         0.000032   
                               xavier uniform 2          0.000028   

                                                  std_val_result  
hidden_size n_layers act_fun   init_methods                       
200         4        ELU       xavier normal  26        0.000812  
                               xavier uniform 10        0.000805  
                     LeakyReLU xavier uniform 5         0.000126  
            6        ELU       xavier normal  4         0.000409  
                               xavier uniform 11        0.000417  
                     ReLU      xavier normal  17        0.000160  
            8        ELU       xavier uniform 28        0.000468  
                     LeakyReLU xavier normal  25        0.000095  
                               xavier uniform 18        0.000061  
400         4        ELU       xavier uniform 13        0.000326  
                     LeakyReLU xavier uniform 30        0.000037  
                     ReLU      xavier uniform 22        0.000039  
            6        ELU       xavier normal  27        0.000313  
                               xavier uniform 12        0.000307  
                     LeakyReLU xavier normal  19        0.000050  
                               xavier uniform 3         0.000030  
            8        ELU       xavier normal  20        0.000359  
                               xavier uniform 24        0.000282  
                     LeakyReLU xavier normal  1         0.000059  
                               xavier uniform 6         0.000029  
                     ReLU      xavier normal  14        0.000069  
                               xavier uniform 9         0.000040  
600         4        LeakyReLU xavier uniform 15        0.000031  
                     ReLU   

In [26]:
crossval_results.to_csv('../results/hyperparams_optimization_res.csv')

## Top 5 Models

- `{'n_hidden': 400, 'n_layers': 8, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'}`
- `{'n_hidden': 400, 'n_layers': 4, 'act_fun': 'ReLU', 'init_method': 'xavier uniform'}`
- `{'n_hidden': 400, 'n_layers': 4, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'}`
- `{'n_hidden': 400, 'n_layers': 6, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'}`
- `{'n_hidden': 400, 'n_layers': 8, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'}`